needed to reload modules, otherwise you will have nan errors

In [1]:
#%load_ext autoreload
#%autoreload 2

### Data

In [2]:
from data_process_and_get_mask import create_mask
from get_data import pbmc_train, pbmc_val

tf_list = ['IRF1', 'IRF2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7',
       'IRF8', 'IRF9', 'STAT1', 'STAT2', 'STAT3', 'STAT4', 'STAT5A',
       'STAT5B','STAT6','GTF2I','RUNX1', 'MYOD1', 'SOX2','KLF4', 'THAP11','ADNP2', 'AEBP1','AHRR', 'ALX4','APBB1','APEX1', 'ARHGAP35','ARID1A']

#['IRF1', 'IRF2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7', 'IRF8', 'IRF9', 'STAT1', 'STAT2', 'STAT3', 'STAT4', 'STAT5A','STAT5B', 'STAT6']
#["GTF2I","GTF3A","NRF1","ELF1","STAT1","STAT2","IRF9","STAT3","STAT4","STAT5A","STAT5B","IRF3","IRF7","IRF1","IRF5","IRF8"]

mask, train, valid = create_mask(pbmc_train, pbmc_val, tf_list)
print("Printing mask:")
print(mask)
print("\nPrinting train:")
print(train)
print("\nPrinting valid:")
print(valid)

Printing mask:
shape: (471, 32)
┌────────┬────────┬────────┬───────┬───┬────────┬───────┬───────┬───────────────┐
│ target ┆ STAT5A ┆ STAT5B ┆ STAT4 ┆ … ┆ ARID1A ┆ ADNP2 ┆ APBB1 ┆ unannotated_1 │
│ ---    ┆ ---    ┆ ---    ┆ ---   ┆   ┆ ---    ┆ ---   ┆ ---   ┆ ---           │
│ str    ┆ f64    ┆ f64    ┆ f64   ┆   ┆ f64    ┆ f64   ┆ f64   ┆ i32           │
╞════════╪════════╪════════╪═══════╪═══╪════════╪═══════╪═══════╪═══════════════╡
│ IL2    ┆ 1.0    ┆ 1.0    ┆ 0.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ IRF1   ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ CCND1  ┆ 1.0    ┆ 1.0    ┆ 0.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ PRF1   ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ IFNG   ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ …      ┆ …      ┆ …      ┆ …     ┆ … ┆ …      ┆ …     ┆ …     ┆ …             │
│ ZNF181 ┆ 0.0    ┆ 0.0    ┆ 0.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 

now use the common training functions

In [3]:
mask

target,STAT5A,STAT5B,STAT4,IRF1,STAT3,RUNX1,GTF2I,IRF2,STAT1,IRF8,STAT6,APEX1,IRF4,MYOD1,ARHGAP35,IRF3,IRF5,ALX4,STAT2,IRF7,KLF4,IRF9,AHRR,AEBP1,SOX2,THAP11,IRF6,ARID1A,ADNP2,APBB1,unannotated_1
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32
"""IL2""",1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""IRF1""",1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""CCND1""",1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
"""PRF1""",1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""IFNG""",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZNF181""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""ZNF230""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""ZNF302""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [5]:
from training.training import run_vega_model
run_vega_model(
    model_type="vega",
    train_data=train,
    valid_data=valid,
    mask_df=mask,
    epochs=30,#30
    N=10,#10
    cond="control"
)

epoch:  0  train_loss:  70.191666   valid_loss:  58.772095
epoch:  10  train_loss:  33.525513   valid_loss:  27.969748
epoch:  20  train_loss:  30.50974   valid_loss:  25.505272
epoch:  0  train_loss:  71.00149   valid_loss:  59.681747
epoch:  10  train_loss:  33.183014   valid_loss:  26.705635
epoch:  20  train_loss:  30.25709   valid_loss:  25.594612
epoch:  0  train_loss:  67.557816   valid_loss:  54.307835
epoch:  10  train_loss:  32.430325   valid_loss:  25.721529
epoch:  20  train_loss:  30.083357   valid_loss:  25.43004
epoch:  0  train_loss:  69.1876   valid_loss:  58.175625
epoch:  10  train_loss:  36.463253   valid_loss:  30.75952
epoch:  20  train_loss:  31.705791   valid_loss:  26.63643
epoch:  0  train_loss:  69.42923   valid_loss:  57.5712
epoch:  10  train_loss:  34.71455   valid_loss:  28.832777
epoch:  20  train_loss:  30.8514   valid_loss:  25.606018
epoch:  0  train_loss:  69.93457   valid_loss:  57.164738
epoch:  10  train_loss:  34.787216   valid_loss:  28.330235
e

(VEGA(
   (decoder): DecoderVEGA(
     (sparse_layer): Sequential(
       (0): SparseLayer()
     )
   )
   (encoder): Encoder(
     (encoder): Sequential(
       (0): Linear(in_features=471, out_features=800, bias=True)
       (1): ReLU()
       (2): Dropout(p=0.3, inplace=False)
       (3): Linear(in_features=800, out_features=800, bias=True)
       (4): ReLU()
       (5): Dropout(p=0.3, inplace=False)
     )
     (mu): Sequential(
       (0): Linear(in_features=800, out_features=31, bias=True)
       (1): Dropout(p=0.5, inplace=False)
     )
     (sigma): Sequential(
       (0): Linear(in_features=800, out_features=31, bias=True)
       (1): Dropout(p=0.5, inplace=False)
     )
   )
 ),
           STAT5A    STAT5B     STAT4      IRF1     STAT3     RUNX1     GTF2I  \
 IL2    -0.000004  0.000214  0.000000  0.000000  0.000168 -0.000086  0.000000   
 IRF1    0.003442  0.008979  0.000055 -0.000071  0.000489  0.000000  0.000000   
 CCND1  -0.000017  0.000094  0.000000  0.000273 -0.000118 